In [128]:
import pandas as pd
import numpy as np
from tqdm.auto import tqdm, trange
import nltk
import re

In [129]:
data = pd.read_csv('../NewsAPI/lenta-ru-news.csv')

In [130]:
topics = ['Путешествия', 'Ценности', 'Мир', 'Наука и техника', 'Экономика']
news_in_cat_count = 400

In [131]:
df_res = pd.DataFrame()

for topic in tqdm(topics):
    df_topic = data[data['topic'] == topic][:news_in_cat_count]
    df_res = df_res.append(df_topic, ignore_index=True)

In [133]:
texts = df_res['text']
type(texts)

pandas.core.series.Series

In [134]:
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("russian")

def token_and_stem(text):
    tokens = [word for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens = []
    for token in tokens:
        if re.search('[а-яА-Я]', token):
            filtered_tokens.append(token)
    stems = [stemmer.stem(t) for t in filtered_tokens]
    return stems

In [135]:
stopwords = nltk.corpus.stopwords.words('russian')
#можно расширить список стоп-слов
stopwords.extend(['что', 'это', 'так', 'вот', 'быть', 'как', 'в', 'к', 'на'])

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

n_featur=200000
tfidf_vectorizer = TfidfVectorizer(max_df=0.8, max_features=10000,
                                 min_df=0.01, stop_words=stopwords,
                                 use_idf=True, tokenizer=token_and_stem, ngram_range=(1,3))


In [136]:
%%time
tfidf_matrix = tfidf_vectorizer.fit_transform(texts)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:385: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['бол', 'больш', 'будт', 'быт', 'вед', 'впроч', 'всег', 'всегд', 'даж', 'друг', 'е', 'ег', 'ем', 'есл', 'ест', 'ещ', 'зач', 'зде', 'ил', 'иногд', 'когд', 'конечн', 'куд', 'лучш', 'межд', 'мен', 'мног', 'мо', 'можн', 'нег', 'нельз', 'нибуд', 'никогд', 'нич', 'опя', 'посл', 'пот', 'почт', 'разв', 'сво', 'себ', 'совс', 'теб', 'тепер', 'тог', 'тогд', 'тож', 'тольк', 'хорош', 'хот', 'чег', 'чут', 'эт'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


Wall time: 23.8 s


In [138]:
type(tfidf_matrix)
tfidf_matrix

<2000x2346 sparse matrix of type '<class 'numpy.float64'>'
	with 152149 stored elements in Compressed Sparse Row format>

In [25]:
print(tfidf_matrix.shape)

(2000, 2346)


In [26]:
tfidf_matrix[:10]

<10x2346 sparse matrix of type '<class 'numpy.float64'>'
	with 674 stored elements in Compressed Sparse Row format>

In [139]:
num_clusters = 5

# Метод к-средних - KMeans
from sklearn.cluster import KMeans
km = KMeans(n_clusters=num_clusters)

In [28]:
%%time
km.fit(tfidf_matrix)

Wall time: 29.6 s


KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
       n_clusters=5, n_init=10, n_jobs=None, precompute_distances='auto',
       random_state=None, tol=0.0001, verbose=0)

In [140]:
%%time
idx = km.fit(tfidf_matrix)
clusters = km.labels_.tolist()

#print(clusters)
#print (km.labels_)

Wall time: 37.8 s


In [146]:
len(km.labels_)
clusters[:10]

[4, 3, 3, 4, 4, 4, 3, 3, 3, 3]

In [147]:
clusterkm = km.labels_.tolist()
frame = pd.DataFrame(texts)

#k-means
out = { 'text': texts, 'cluster': clusterkm, 'topic': df_res['topic'] }
frame1 = pd.DataFrame(out, columns = ['text', 'cluster', 'topic'])

In [150]:
frame1.head(10)

,text,cluster,topic
0,Сотрудники социальной сети Instagram проанализ...,4,Путешествия
1,Пассажирский самолет российской авиакомпании «...,3,Путешествия
2,Пассажиры самолета авиакомпании «Уральские ави...,3,Путешествия
3,Депутат Госдумы от ЛДПР Василий Власов предлож...,4,Путешествия
4,Россиянин умер в международном аэропорту Дабол...,4,Путешествия
5,"Египтяне арестованы за то, что помогли туриста...",4,Путешествия
6,Пассажир авиакомпании Spirit Airlines домогалс...,3,Путешествия
7,Самолет авиакомпании Southwest Airlines вернул...,3,Путешествия
8,Пассажиры ирландской авиакомпании Ryanair устр...,3,Путешествия
9,"Самолет с российскими туристами, которые не мо...",3,Путешествия


In [153]:
fp = frame1[ frame1.topic.eq('Путешествия') & frame1.cluster.eq(2) ]
print(len(fp))

7


In [69]:
# MiniBatchKMeans
from sklearn.cluster import MiniBatchKMeans

mbk  = MiniBatchKMeans(init='random', n_clusters=num_clusters) #(init='k-means++', ‘random’ or an ndarray)
mbk.fit_transform(tfidf_matrix)
%time mbk.fit(tfidf_matrix)
miniclusters = mbk.labels_.tolist()
print (mbk.labels_)

Wall time: 52.9 ms
[4 2 2 ... 3 4 1]


In [ ]:
ser = data[ data['topic'] == 'Путешествия']['title']

In [109]:
print(topics)
print(num_clusters)

['Путешествия', 'Ценности', 'Мир', 'Наука и техника', 'Экономика']
5


In [113]:
for topic in topics:
    print(topic, end = ' \t\t ')
    for cluster in range(num_clusters):
        print(cluster, ' : ', len(frame1[ frame1.topic.eq(topic) &  frame1.cluster.eq(cluster) ]), end = ' ')
    print()

Путешествия 		 0  :  7 1  :  10 2  :  36 3  :  192 4  :  155 
Ценности 		 0  :  296 1  :  7 2  :  8 3  :  88 4  :  1 
Мир 		 0  :  0 1  :  215 2  :  10 3  :  157 4  :  18 
Наука и техника 		 0  :  0 1  :  107 2  :  14 3  :  227 4  :  52 
Экономика 		 0  :  1 1  :  46 2  :  215 3  :  131 4  :  7 


In [154]:
clust_res = pd.DataFrame(columns=['topic', 'cluster_0', 'cluster_1', 'cluster_2', 'cluster_3', 'cluster_4'])
i = 0
for topic in topics:
    cnt = []
    for cluster in range(num_clusters):
        cnt.append(len(frame1[ frame1.topic.eq(topic) &  frame1.cluster.eq(cluster) ]))
    clust_res.loc[i] = [topic] + cnt
    i += 1
clust_res

,topic,cluster_0,cluster_1,cluster_2,cluster_3,cluster_4
0,Путешествия,10,40,7,165,178
1,Ценности,300,0,7,1,92
2,Мир,0,24,184,11,181
3,Наука и техника,0,208,12,2,178
4,Экономика,0,0,72,10,318


In [155]:
tech_text = '''
Ученые Университета Аризоны в США открыли неизвестный принцип эволюции, согласно которому естественный отбор не может благоприятствовать сразу нескольким полезным мутациям, повышающим приспособленность организма к окружающей среде. Вместо этого он фокусируется только на том, что достаточно для выживания существа, и игнорирует другие возможности для улучшения. Об этом сообщается в статье в журнале Proceedings of the National Academy of Sciences.
Исследователи спровоцировали несколько мутаций, которые «ломали» механизм трансляции у шести различных штаммов бактерии кишечной палочки Escherichia coli. Трансляция — это процесс, при котором заложенная в генах информация о нуклеотидной последовательности белков переносится к рибосомам и участвует в белковом синтезе. Известно, что более чем за 3,5 миллиарда лет эволюции механизм трансляции не претерпел существенных изменений.
Ряд нефатальных мутаций, затрагивающих функционирование трансляционного процесса, должен с течением времени быть исправлен эволюцией. Для этого ученые заставили штаммы бактерий с различными изменениями конкурировать друг с другом, благодаря чему осуществлялся естественный отбор. Со временем внутри популяции накапливались полезные мутации, которые могли быть сохранены и использованы для улучшения трансляции.'''

In [156]:
text_vec = tfidf_vectorizer.transform([tech_text])
pd.DataFrame.sparse.from_spmatrix(text_vec)

,0,1,2,3,4,5,6,7,8,9,...,2336,2337,2338,2339,2340,2341,2342,2343,2344,2345
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [158]:
km.predict(text_vec)

array([4])

In [159]:
econ_text = '''
Доллар США считается символом надежности, безопасности и экономического процветания. 
Он занимает неоспоримое доминирующее положение в международной финансовой системе с середины XX века и производит 
впечатление непобедимого титана. Однако эра господства доллара как основной мировой резервной валюты медленно 
подходит к концу. Крупнейшие банки предрекают ему резкий спад уже в следующем году, а известный экономист 
Стивен Роуч уверен, что американская валюта может обесцениться на треть. 
Причинами обвала станут сокращение сбережений населения, рост государственного долга США и усиление Китая. 
'''

In [160]:
text_vec = tfidf_vectorizer.transform([econ_text])
km.predict(text_vec)

array([4])

## Меры сходства 

In [161]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from sklearn.metrics.pairwise import cosine_similarity
 
# зададим массив текстов
some_texts = [
   'текст номер один',
   'текст номер два',
   'текст следующий под номером три',
]
df = pd.DataFrame({'texts': some_texts})

In [162]:
find_nearest_to = "документ номер два"

In [163]:
# формирование весов tf-idf
tfidf = TfidfVectorizer()
mx_tf = tfidf.fit_transform(some_texts)
new_entry = tfidf.transform([find_nearest_to])

In [164]:
type(mx_tf)

scipy.sparse.csr.csr_matrix

In [98]:
sdf = pd.DataFrame.sparse.from_spmatrix(mx_tf)
sdf

,0,1,2,3,4,5,6,7
0,0.000000,0.547832,0.000000,0.720333,0.000000,0.000000,0.425441,0.000000
1,0.720333,0.547832,0.000000,0.000000,0.000000,0.000000,0.425441,0.000000
2,0.000000,0.000000,0.479528,0.000000,0.479528,0.479528,0.283217,0.479528


In [165]:
cosine_similarity(mx_tf)

array([[1.        , 0.48111972, 0.12049196],
       [0.48111972, 1.        , 0.12049196],
       [0.12049196, 0.12049196, 1.        ]])

In [104]:
new_entry = tfidf.transform([find_nearest_to])
pd.DataFrame.sparse.from_spmatrix(new_entry)

,0,1,2,3,4,5,6,7
0,0.795961,0.605349,0.0,0.0,0.0,0.0,0.0,0.0


In [166]:
cosine_similarities = cosine_similarity(new_entry, mx_tf).flatten()
cosine_similarities

array([0.33162938, 0.90498638, 0.        ])

In [167]:
# запишем все попарные результаты сравнений
df['cos_similarities'] = cosine_similarities
# и отсортируем по убыванию (т.к. cos(0)=1)
df = df.sort_values(by=['cos_similarities'], ascending=[0])
df

,texts,cos_similarities
1,текст номер два,0.904986
0,текст номер один,0.331629
2,текст следующий под номером три,0.000000


Полный список метрик попарного сравнения - в документации https://scikit-learn.org/stable/modules/classes.html#module-sklearn.metrics.pairwise

## Визуализация

In [65]:
from matplotlib import rc
#включаем русские символы на графике
font = {'family' : 'Verdana'}#, 'weigth': 'normal'}
rc('font', **font)

In [68]:
for name, group in groups_mbk:
    points = ax.plot(group.x, group.y, marker='o', linestyle='', ms=12, #ms=18
                     label=cluster_names[name], mec='none', 
                     color=cluster_colors[name])
    ax.set_aspect('auto')
    labels = [i for i in group.title]
    
    tooltip = mpld3.plugins.PointHTMLTooltip(points[0], labels, voffset=10, hoffset=10) #, css=css
    mpld3.plugins.connect(fig, tooltip) #   , TopToolbar()
    
    #ax.axes.get_xaxis().set_ticks([])
    #ax.axes.get_yaxis().set_ticks([])

    #ax.axes.get_xaxis().set_visible(False)
    #ax.axes.get_yaxis().set_visible(False)

NameError: name 'groups_mbk' is not defined

In [ ]:
ax.set_title("Mini K-Means", size=20) #groups_mbk
    
ax.legend(numpoints=1)
mpld3.disable_notebook()
#mpld3.display()
mpld3.save_html(fig, "mbk.html")

mpld3.show()
#mpld3.save_json(fig, "vivod.json")
#mpld3.fig_to_html(fig)

fig, ax = plt.subplots(figsize=(51,25))

scatter = ax.scatter(np.random.normal(size=N),
                     np.random.normal(size=N),
                     c=np.random.random(size=N),
                     s=1000 * np.random.random(size=N),
                     alpha=0.3,
                     cmap=plt.cm.jet)
ax.grid(color='white', linestyle='solid')    

ax.set_title("Кластеры", size=20)

fig, ax = plt.subplots(figsize=(51,25))

labels = ['point {0}'.format(i + 1) for i in range(N)]
tooltip = mpld3.plugins.PointLabelTooltip(scatter, labels=labels)
mpld3.plugins.connect(fig, tooltip)

mpld3.show()fig, ax = plt.subplots(figsize=(72,36))
for name, group in groups:
    points = ax.plot(group.x, group.y, marker='o', linestyle='', ms=18, 
                     label=cluster_names[name], mec='none', 
                     color=cluster_colors[name])
    ax.set_aspect('auto')
    labels = [i for i in group.title]
    tooltip = mpld3.plugins.PointLabelTooltip(points, labels=labels)
    mpld3.plugins.connect(fig, tooltip)

ax.set_title("K-means", size=20)

mpld3.display()